In [ ]:
import numpy as np

In [ ]:
import pandas as pd

# Define the path to the master CSV
master_file = "---" #put the master CSV file path here

# Load the CSV file into a DataFrame
master_df = pd.read_csv(master_file)

# Display basic info about the dataset
print("Data Loaded Successfully!")
print(master_df.info())  # Check column names and data types
print(master_df.head())  # Show first few rows

<ipython-input-2-04d960e4bf4f>:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  master_df = pd.read_csv(master_file)


Data Loaded Successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669747 entries, 0 to 669746
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SYMBOL          669747 non-null  object 
 1    SERIES         669747 non-null  object 
 2    DATE1          669747 non-null  object 
 3    PREV_CLOSE     669747 non-null  float64
 4    OPEN_PRICE     669747 non-null  float64
 5    HIGH_PRICE     669747 non-null  float64
 6    LOW_PRICE      669747 non-null  float64
 7    LAST_PRICE     669747 non-null  object 
 8    CLOSE_PRICE    669747 non-null  float64
 9    AVG_PRICE      669747 non-null  float64
 10   TTL_TRD_QNTY   669747 non-null  int64  
 11   TURNOVER_LACS  669747 non-null  float64
 12   NO_OF_TRADES   669747 non-null  int64  
 13   DELIV_QTY      669747 non-null  object 
 14   DELIV_PER      669747 non-null  object 
dtypes: float64(7), int64(2), object(6)
memory usage: 76.6+ MB
None
       SYMBOL

In [ ]:
# Selecting only the required columns
deliv_df = master_df[["SYMBOL", " SERIES", " DATE1", " DELIV_QTY"]]

# Display first few rows of the filtered data
print(deliv_df.head())

       SYMBOL  SERIES         DATE1  DELIV_QTY
0  1018GS2026      GS   01-Apr-2024        119
1   20MICRONS      EQ   01-Apr-2024      38325
2  21STCENMGM      BE   01-Apr-2024          -
3      360ONE      EQ   01-Apr-2024     131752
4   3IINFOLTD      EQ   01-Apr-2024     750968


In [ ]:
# Filter the DataFrame where SERIES is 'EQ'
eq_deliv_df = deliv_df[deliv_df[" SERIES"] == " EQ"]

# Display first few rows
print(eq_deliv_df.head())

       SYMBOL  SERIES         DATE1  DELIV_QTY
1   20MICRONS      EQ   01-Apr-2024      38325
3      360ONE      EQ   01-Apr-2024     131752
4   3IINFOLTD      EQ   01-Apr-2024     750968
5     3MINDIA      EQ   01-Apr-2024       2246
11     5PAISA      EQ   01-Apr-2024      44930


In [ ]:
num_stocks = eq_deliv_df["SYMBOL"].nunique()
print(f"Total unique stocks in EQ series: {num_stocks}")

Total unique stocks in EQ series: 2331


In [ ]:
# sorting the stocks first by name and then by date
eq_deliv_df.loc[:, " DATE1"] = pd.to_datetime(eq_deliv_df[" DATE1"])
sorted_df = eq_deliv_df.sort_values(by=["SYMBOL", " DATE1"])
print(sorted_df.head())

          SYMBOL  SERIES                DATE1  DELIV_QTY
1      20MICRONS      EQ  2024-04-01 00:00:00      38325
2670   20MICRONS      EQ  2024-04-02 00:00:00     115203
5329   20MICRONS      EQ  2024-04-03 00:00:00      29032
8006   20MICRONS      EQ  2024-04-04 00:00:00      38282
10655  20MICRONS      EQ  2024-04-05 00:00:00      16471


In [ ]:
# Ensure DATE column is in datetime format
sorted_df[" DATE1"] = pd.to_datetime(sorted_df[" DATE1"])

# Create 'WEEK' and 'YEAR' columns
#sorted_df["WEEK"] = sorted_df[" DATE1"].dt.to_period("W")
#sorted_df["YEAR"] = sorted_df[" DATE1"].dt.year

In [ ]:
# Define the Monday of each week (to group by trading weeks)
sorted_df["WEEK"] = sorted_df[" DATE1"].apply(lambda x: x - pd.Timedelta(days=x.weekday()))

# Extract the year for yearly grouping
sorted_df["YEAR"] = sorted_df[" DATE1"].dt.year

In [ ]:
# Convert DELIV_QTY to numeric (force errors='coerce' to handle any bad data)
sorted_df[" DELIV_QTY"] = pd.to_numeric(sorted_df[" DELIV_QTY"], errors="coerce")

# Now perform groupby aggregation correctly
weekly_deliv = sorted_df.groupby(["SYMBOL", "WEEK"])[" DELIV_QTY"].sum().reset_index()
yearly_deliv = sorted_df.groupby(["SYMBOL", "YEAR"])[" DELIV_QTY"].sum().reset_index()


In [ ]:
print(weekly_deliv.tail())  # Check column names in weekly data
print(yearly_deliv.head())  # Check column names in yearly data

           SYMBOL       WEEK  DELIV_QTY_WEEK
100999  ZYDUSWELL 2025-02-24           41696
101000  ZYDUSWELL 2025-03-03          244667
101001  ZYDUSWELL 2025-03-10          147237
101002  ZYDUSWELL 2025-03-17          121609
101003  ZYDUSWELL 2025-03-24           90942
      SYMBOL  YEAR   DELIV_QTY
0  20MICRONS  2024    21038666
1  20MICRONS  2025     4151045
2     360ONE  2024    77055772
3     360ONE  2025    31556159
4  3IINFOLTD  2024    85385050


In [ ]:
print(weekly_deliv.shape[0])
print(yearly_deliv.shape[0])

101004
4464


In [ ]:
unique_stocks = yearly_deliv['SYMBOL'].nunique()
print("Unique stocks in yearly_deliv:", unique_stocks)

Unique stocks in yearly_deliv: 2331


In [ ]:
# Aggregate the deliverable quantities by SYMBOL (summing for 2024 and 2025)
aggr_yearly_deliv = yearly_deliv.groupby("SYMBOL", as_index=False)[" DELIV_QTY"].sum()

# Display the result
print(aggr_yearly_deliv)

         SYMBOL   DELIV_QTY
0     20MICRONS    25189711
1        360ONE   108611931
2     3IINFOLTD   116149947
3       3MINDIA      772110
4        3PLAND     4149796
...         ...         ...
2326       ZOTA     7085013
2327      ZUARI    32705801
2328   ZUARIIND    15550542
2329  ZYDUSLIFE   210717051
2330  ZYDUSWELL     6384519

[2331 rows x 2 columns]


In [ ]:
# Rename columns properly
weekly_deliv.columns = ["SYMBOL", "WEEK", "DELIV_QTY_WEEK"]
aggr_yearly_deliv.columns = ["SYMBOL", "DELIV_QTY_YEAR"]

In [ ]:
# Count the number of trading days for each stock
trading_days = sorted_df.groupby(["SYMBOL", "YEAR"])[" DATE1"].nunique().reset_index()
trading_days_agg = trading_days.groupby("SYMBOL", as_index=False)[" DATE1"].sum()
print(trading_days_agg.head())

      SYMBOL   DATE1
0  20MICRONS     182
1     360ONE     247
2  3IINFOLTD     247
3    3MINDIA     247
4     3PLAND     102


In [ ]:
trading_days_agg.rename(columns={" DATE1": "TRADING_DAYS"}, inplace=True)

In [ ]:
weeks_count = weekly_deliv.groupby("SYMBOL").size().reset_index(name="WEEKS_TRADED")
print(weeks_count)

         SYMBOL  WEEKS_TRADED
0     20MICRONS            39
1        360ONE            52
2     3IINFOLTD            52
3       3MINDIA            52
4        3PLAND            23
...         ...           ...
2326       ZOTA            52
2327      ZUARI            52
2328   ZUARIIND            52
2329  ZYDUSLIFE            52
2330  ZYDUSWELL            52

[2331 rows x 2 columns]


In [ ]:
combined_df = aggr_yearly_deliv.merge(trading_days_agg, on="SYMBOL", how="outer")

In [ ]:
combined_df = combined_df.merge(weeks_count, on="SYMBOL", how="outer")
print(combined_df)

         SYMBOL  DELIV_QTY_YEAR  TRADING_DAYS  WEEKS_TRADED
0     20MICRONS        25189711           182            39
1        360ONE       108611931           247            52
2     3IINFOLTD       116149947           247            52
3       3MINDIA          772110           247            52
4        3PLAND         4149796           102            23
...         ...             ...           ...           ...
2326       ZOTA         7085013           247            52
2327      ZUARI        32705801           247            52
2328   ZUARIIND        15550542           247            52
2329  ZYDUSLIFE       210717051           247            52
2330  ZYDUSWELL         6384519           247            52

[2331 rows x 4 columns]


In [ ]:
# AVG_WEEKLY_DELIV_QTY, CURRENT_WEEKLY_DELIV

In [ ]:
# weekly_deliv has DELIV_QTY for every week for every stock
# weeks_count has no. of weeks every stock has traded for
# create a new dataframe compare_df
# get AVG_WEEKLY_DEVLIV_QTY by dividing DELIV_QTY_YEAR with WEEKS_TRADED and add AVG_WEEKLY_DEVLIV_QTY as a column in compare_df
# get CURRENT_WEEKLY_DELIV in another column by taking the latest weekly deliv from weekly_deliv (CURRENT WEEK'S DELIV).

In [ ]:
# Calculate AVG_WEEKLY_DEVLIV_QTY in combined_df
combined_df['AVG_WEEKLY_DEVLIV_QTY'] = (combined_df['DELIV_QTY_YEAR'] / combined_df['WEEKS_TRADED']).round(2)

# Extract the latest weekly deliverable quantity for each stock from weekly_deliv.
# Sort by WEEK in descending order, then group by SYMBOL and take the first row.
latest_weekly = weekly_deliv.sort_values(by="WEEK", ascending=False).groupby("SYMBOL").first().reset_index()

# Rename the deliverable quantity column from weekly_deliv to CURRENT_WEEKLY_DELIV
latest_weekly = latest_weekly.rename(columns={"DELIV_QTY_WEEK": "CURRENT_WEEKLY_DELIV"})

# Merge the average weekly value from combined_df with the latest weekly deliverable from latest_weekly.
compare_df = pd.merge(combined_df[['SYMBOL', 'AVG_WEEKLY_DEVLIV_QTY']],
                      latest_weekly[['SYMBOL', 'CURRENT_WEEKLY_DELIV']],
                      on="SYMBOL", how="left")

# Display the final compare_df
print(compare_df)

         SYMBOL  AVG_WEEKLY_DEVLIV_QTY  CURRENT_WEEKLY_DELIV
0     20MICRONS              645890.03                367750
1        360ONE             2088690.98               3082938
2     3IINFOLTD             2233652.83               5073282
3       3MINDIA               14848.27                 14081
4        3PLAND              180425.91                 71120
...         ...                    ...                   ...
2326       ZOTA              136250.25                 95224
2327      ZUARI              628957.71                525809
2328   ZUARIIND              299048.88                298418
2329  ZYDUSLIFE             4052250.98               4341568
2330  ZYDUSWELL              122779.21                 90942

[2331 rows x 3 columns]


In [ ]:
# Create a new column "X_TIMES" in compare_df, which is CURRENT_WEEKLY_DELIV expressed as X times of AVG_WEEKLY_DEVLIV_QTY
compare_df["X_TIMES"] = (compare_df["CURRENT_WEEKLY_DELIV"] / compare_df["AVG_WEEKLY_DEVLIV_QTY"]).round(2)

# Display the updated DataFrame
print(compare_df)

         SYMBOL  AVG_WEEKLY_DEVLIV_QTY  CURRENT_WEEKLY_DELIV WEEK_TRIGGER  \
0     20MICRONS              645890.03                367750           NO   
1        360ONE             2088690.98               3082938          YES   
2     3IINFOLTD             2233652.83               5073282          YES   
3       3MINDIA               14848.27                 14081           NO   
4        3PLAND              180425.91                 71120           NO   
...         ...                    ...                   ...          ...   
2326       ZOTA              136250.25                 95224           NO   
2327      ZUARI              628957.71                525809           NO   
2328   ZUARIIND              299048.88                298418           NO   
2329  ZYDUSLIFE             4052250.98               4341568          YES   
2330  ZYDUSWELL              122779.21                 90942           NO   

      X_TIMES  
0        0.57  
1        1.48  
2        2.27  
3        0.

In [ ]:
# Add the WEEK_TRIGGER column: "YES" if CURRENT_WEEKLY_DELIV > AVG_WEEKLY_DEVLIV_QTY, otherwise "NO"
compare_df["WEEK_TRIGGER"] = np.where(compare_df["CURRENT_WEEKLY_DELIV"] > compare_df["AVG_WEEKLY_DEVLIV_QTY"], "YES", "NO")

# Display the final compare_df
compare_df

,SYMBOL,AVG_WEEKLY_DEVLIV_QTY,CURRENT_WEEKLY_DELIV,WEEK_TRIGGER,X_TIMES
0,20MICRONS,645890.03,367750,NO,0.57
1,360ONE,2088690.98,3082938,YES,1.48
2,3IINFOLTD,2233652.83,5073282,YES,2.27
3,3MINDIA,14848.27,14081,NO,0.95
4,3PLAND,180425.91,71120,NO,0.39
...,...,...,...,...,...
2326,ZOTA,136250.25,95224,NO,0.70
2327,ZUARI,628957.71,525809,NO,0.84
2328,ZUARIIND,299048.88,298418,NO,1.00
2329,ZYDUSLIFE,4052250.98,4341568,YES,1.07


In [ ]:
yes_stocks = compare_df[compare_df["WEEK_TRIGGER"] == "YES"]
yes_stocks

,SYMBOL,AVG_WEEKLY_DEVLIV_QTY,CURRENT_WEEKLY_DELIV,WEEK_TRIGGER,X_TIMES
1,360ONE,2.088691e+06,3082938,YES,1.48
2,3IINFOLTD,2.233653e+06,5073282,YES,2.27
5,5PAISA,1.575178e+05,198062,YES,1.26
8,AAATECH,1.542754e+05,164354,YES,1.07
13,AARTECH,5.534617e+05,1817084,YES,3.28
...,...,...,...,...,...
2322,ZIMLAB,5.642485e+05,649823,YES,1.15
2323,ZODIAC,7.635489e+04,85016,YES,1.11
2324,ZODIACLOTH,1.197851e+05,120574,YES,1.01
2325,ZOMATO,1.326903e+08,446561380,YES,3.37


In [ ]:
yes_stocks = yes_stocks.sort_values(by="X_TIMES", ascending=False)
yes_stocks

,SYMBOL,AVG_WEEKLY_DEVLIV_QTY,CURRENT_WEEKLY_DELIV,WEEK_TRIGGER,X_TIMES
1022,JBCHEPHARM,695161.54,9379744,YES,13.49
767,GSEC10ABSL,44500.63,582312,YES,13.09
1101,KESORAMIND,2666077.68,26852287,YES,10.07
1536,ORIENTBELL,65252.21,611979,YES,9.38
395,CGCL,900235.38,8433387,YES,9.37
...,...,...,...,...,...
2088,TCS,7077926.67,7087351,YES,1.00
1432,NDRAUTO,63118.52,63153,YES,1.00
607,ESSENTIA,20673418.95,20737674,YES,1.00
1699,RANEHOLDIN,67760.15,67932,YES,1.00


In [ ]:
# now, we add another feature to this.
# the user should be allowed to type a stock ticker (symbol) and the output should be if that stock has triggered or not.
# also return the X_TIMES value for that stock.

In [ ]:
def check_stock_trigger(stock_symbol):
    # Filter compare_df for the given stock symbol
    stock_data = compare_df[compare_df["SYMBOL"] == stock_symbol]

    if stock_data.empty:
        print(f"No data found for symbol {stock_symbol}.")
    else:
        # Assuming one row per stock in compare_df:
        row = stock_data.iloc[0]
        trigger_status = row["WEEK_TRIGGER"]
        xtimes = row["X_TIMES"]
        print(f"Stock {stock_symbol} Triggered: {trigger_status}")
        print(f"X_TIMES value: {xtimes}")

# Take input outside the function
user_input = input("Enter stock symbol: ").strip().upper()
check_stock_trigger(user_input)

Enter stock symbol: moil
Stock MOIL Triggered: NO
X_TIMES value: 0.61


In [ ]:
# ver 2.0